In [8]:
from pathlib import Path
if Path.cwd().parent.stem == 't2':
    %cd ..
%config Completer.use_jedi = False

## Seed

In [2]:
import torch
import pytorch_lightning as pl

pl.seed_everything(hash("setting a random seeds") % 2**32 - 1)
torch.backends.cudnn.benchmark = True

Global seed set to 3397482767


In [9]:
from torch.utils.data import random_split, DataLoader, Dataset
from torchvision import transforms, datasets
import skimage.morphology as morph

class ErosionReplicate(object):
    """Applies Erosion to one channel and replicates the channel"""
    def __call__(self, tensor):
        one_channel = tensor[0, :, :]
        one_channel = torch.from_numpy(morph.erosion(one_channel, morph.square(3)))    
        tensor[0, :, :] = one_channel
        tensor[1, :, :] = one_channel
        tensor[2, :, :] = one_channel
        return tensor
        

class SiameseDataset(Dataset):
    def __init__(self, base_dir_sk: str,  base_dir_ph: str, triplet: bool = False):
        super(Dataset, self).__init__()
        
        # base_dirs
        self.base_dir_sk = str(Path(base_dir) / 'png_w256')
        self.base_dir_ph = base_dir 
        
        # transforms
        base_tr = [
            transforms.Resize([224,224]),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
        ]
        sketch_tr = [ErosionReplicate()]
        normalize_tr = [transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5))]
        
        self.transforms_sk = transforms.Compose(base_tr + sketch_tr + normalize_tr)
        self.transforms_ph = transforms.Compose(base_tr + normalize_tr)
        
        
        # init two ImageFolders        
        self.dataset_sk = datasets.ImageFolder(self.base_dir_sk)
        self.dataset_ph = datasets.ImageFolder(self.base_dir_ph)
        
        self.n_classes = len(self.classes)
        self.idx_to_class = {idx: label for label, idx in self.class_to_idx.items()}
        
    self.base_dir, transform = self.transforms
    
    
    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        """
        Args:
            index (int): Index
        Returns:
            tuple: (sample, target) where target is class_index of the target class.
        """
        path_sk, target_sk = self.samples[index]
        sample = self.loader(path)
        
        if self.transform is not None:
            sample = self.transform(sample)
            
        if self.target_transform is not None:
            target = self.target_transform(target)

        return sample, target

NameError: name 'self' is not defined

In [8]:
SimpleDataset('data/Flickr15K/', 'photo').samples

[('data/Flickr15K/images/1/1001395.jpg', 0),
 ('data/Flickr15K/images/1/100246.jpg', 0),
 ('data/Flickr15K/images/1/1004885.jpg', 0),
 ('data/Flickr15K/images/1/1006701.jpg', 0),
 ('data/Flickr15K/images/1/1006751.jpg', 0),
 ('data/Flickr15K/images/1/1006928.jpg', 0),
 ('data/Flickr15K/images/1/1008297.jpg', 0),
 ('data/Flickr15K/images/1/1009042.jpg', 0),
 ('data/Flickr15K/images/1/1009216.jpg', 0),
 ('data/Flickr15K/images/1/1009572.jpg', 0),
 ('data/Flickr15K/images/1/1009886.jpg', 0),
 ('data/Flickr15K/images/1/101094.jpg', 0),
 ('data/Flickr15K/images/1/101097.jpg', 0),
 ('data/Flickr15K/images/1/1011324.jpg', 0),
 ('data/Flickr15K/images/1/1011369.jpg', 0),
 ('data/Flickr15K/images/1/1012301.jpg', 0),
 ('data/Flickr15K/images/1/101719.jpg', 0),
 ('data/Flickr15K/images/1/1020889.jpg', 0),
 ('data/Flickr15K/images/1/1022403.jpg', 0),
 ('data/Flickr15K/images/1/1022437.jpg', 0),
 ('data/Flickr15K/images/1/102256.jpg', 0),
 ('data/Flickr15K/images/1/1022711.jpg', 0),
 ('data/Flickr1

Contrastive Model        |  Triplet Model
:-------------------------:|:-------------------------:
![alt text](../informe/contrastive_arch.png "Title") | ![alt text](../informe/triplet_arch.png "Title")

In [7]:
from torch.nn import functional as F

class Normalize(nn.Module):
    def __init__(self, p: float = 2, dim: int = 1, eps: float = 1e-12):
        self.p = p
        self.dim = dim
        self.eps = eps
        
    def forward(self, input: Tensor):
        return F.normalize(input, self.p, self.dim, self.eps)


class SimpleLearner(pl.LightningModule):
    def __init__(self, models: Type[Union[nn.Module, dict]], n_classes: int, mode: str):
        super().__init__()
        
        # shared weights
        shared_mlp = nn.Sequential(
            nn.Linear(model.fc.in_features, 512), # fc1
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 512) # fc2
        )
        
        # shared weights for classification
        classifier_mlp = nn.Sequential(
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, n_classes)
        )
        
        # normalization before contrastive or triplet
        siamese_out = nn.Sequential(
            nn.Normalize()
        )

        self.ce_loss = F.cross_entropy
        
        if mode == 'contrastive':
            try:
                self.criterions = {
                    'anchor': self.ce_loss,
                    'positive': self.ce_loss,
                    'constrative': ContrastiveLoss()
                }
            except NameError:
                raise NotImplementedError('Contrastive Loss Not Implemented')

        elif mode == 'triplet':
            triplet_loss = F.triplet_margin_loss
            
            self.criterions = {
                'anchor': self.ce_loss,
                'positive': self.ce_loss,
                'negative': self.ce_loss,
                'triplet':  triplet_loss
            }

    def forward(self, x):
        if mode == 'contrastive':
            # anchor is sketch, positive is photo
            x_an, x_pos = x
            emb_an = models['sketch'](x_an)
            emb_pos = models['photo'](x_pos)
        
        elif mode == 'triplet':
            # anchor is sketch, positive and negative is photo
            x_anc, x_pos, x_neg = x
            emb_anc = shared_mlp(models['sketch'](x_anc))
            emb_pos = shared_mlp(models['photo'](x_pos))
            emb_neg = shared_mlp(models['photo'](x_pos))
            
            n_emb_anc = siamese_out(emb_anc)
            n_emb_pos = siamese_out(emb_pos)
            n_emb_neg = siamese_out(emb_neg)
            
                        
            y_anc = classifier_mlp(emb_anc)
            y_pos = classifier_mlp(emb_pos)
            y_neg = classifier_mlp(emb_neg)
            
            
            return (n_emb_anc, n_emb_pos, n_emb_neg), (y_anc, y_pos, y_neg)

    def training_step(self, batch,  batch_ixd):
        samples, targets = batch[0], batch[1]
        
        embeddings, y_hats = self.forward(samples)
        
        loss = self.criterion(img_sr, img_hr)
        
        self.log('train_loss', loss)
        return {'loss': loss}

    def validation_step(self, batch, batch_ixd):
        img_lr = batch['lr']
        img_hr = batch['hr']
        img_sr = self.forward(img_lr)
        
        val_loss = self.criterion(img_sr, img_hr)
        
        self.log('val_loss', val_loss)
        return {'val_loss':val_loss }

    def configure_optimizers(self):
        # you define one or multiple optimizer, learning rate schedulers, etc.
        # links provided on the train.py about setting up 2 opts for GAN
        return [optim.Adam(self.parameters(), lr=PARAMS['learning_rate'])]


SyntaxError: invalid syntax (<ipython-input-7-400bf5f10460>, line 10)